In [120]:
!pip install datasets
!pip install gensim
!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.8 MB/s eta 0:00:0000:0100:01


In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [5]:

print(test_dataset.to_pandas().head(15))

print(test_dataset[:15])  # View the first 5 rows as a dictionary


                                                 text  label
0   lovingly photographed in the manner of a golde...      1
1               consistently clever and suspenseful .      1
2   it's like a " big chill " reunion of the baade...      1
3   the story gives ample opportunity for large-sc...      1
4                   red dragon " never cuts corners .      1
5   fresnadillo has something serious to say about...      1
6   throws in enough clever and unexpected twists ...      1
7   weighty and ponderous but every bit as filling...      1
8   a real audience-pleaser that will strike a cho...      1
9   generates an enormous feeling of empathy for i...      1
10  exposing the ways we fool ourselves is one hou...      1
11  it's up to you to decide whether to admire the...      1
12  mostly , [goldbacher] just lets her complicate...      1
13  . . . quite good at providing some good old fa...      1
14  at its worst , the movie is pretty diverting ;...      1
{'text': ['lovingly phot

# Google Word2Vec

In [9]:
import os
import gensim.downloader as api
from gensim.models import KeyedVectors

# Define the model path
model_name = "word2vec-google-news-300"

path = api.load(model_name, return_path=True)
print(f"Model downloaded and saved at: {path}")


# Load the model
word2vec_model = KeyedVectors.load_word2vec_format(path, binary=True)
print("Model loaded successfully.")


Model downloaded and saved at: /Users/akhi/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
Model loaded successfully.


In [131]:
# Find words similar to "king"
similar_words = word2vec_model.most_similar("king", topn=10)
print("Words similar to 'king':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

Words similar to 'king':
kings: 0.7138
queen: 0.6511
monarch: 0.6413
crown_prince: 0.6204
prince: 0.6160
sultan: 0.5865
ruler: 0.5798
princes: 0.5647
Prince_Paras: 0.5433
throne: 0.5422


In [132]:
# Compute similarity between "king" and "queen"
similarity = word2vec_model.similarity("king", "queen")
print(f"Similarity between 'king' and 'queen': {similarity:.4f}")

Similarity between 'king' and 'queen': 0.6511


In [133]:
# Get the vector for the word "apple"
vector = word2vec_model["an"]
print("Vector for 'apple':", vector)

Vector for 'apple': [ 0.12597656  0.19042969  0.06982422  0.07226562 -0.21875     0.02758789
  0.05395508 -0.07666016  0.04296875 -0.12988281  0.05151367 -0.10498047
 -0.11474609  0.05566406 -0.05029297 -0.00424194  0.08447266  0.00915527
 -0.09277344 -0.02770996  0.08935547 -0.11425781 -0.13476562 -0.09521484
 -0.10742188 -0.10888672 -0.02563477  0.31054688  0.17382812  0.19921875
  0.0090332   0.02905273 -0.09863281 -0.10058594 -0.21191406 -0.11962891
 -0.06542969  0.125      -0.07617188 -0.11181641 -0.19726562  0.12695312
  0.02770996  0.04785156  0.11035156  0.04760742  0.01330566  0.19726562
  0.11962891  0.04296875  0.07861328 -0.07128906  0.21972656  0.00772095
 -0.16210938  0.125       0.10498047 -0.03588867 -0.03710938 -0.11767578
 -0.29296875  0.11669922  0.04077148 -0.00152588 -0.06445312  0.24023438
  0.29101562 -0.03198242 -0.02160645  0.08398438  0.0703125   0.05175781
  0.17871094 -0.05639648  0.09472656 -0.01275635  0.02050781  0.13867188
 -0.00069427  0.26757812  0.174

# GLOVE Embedding Model

In [137]:
glove_model = api.load("glove-wiki-gigaword-300")
print("GloVe model loaded successfully.")

GloVe model loaded successfully.


In [138]:
# Step 2: Testing the GloVe model

# 1. Check similar words
print("Words similar to 'king':")
similar_words = glove_model.most_similar("king", topn=10)
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

# 2. Compute word similarity
similarity = glove_model.similarity("king", "queen")
print(f"\nSimilarity between 'king' and 'queen': {similarity:.4f}")

# 3. Perform analogy
analogy_result = glove_model.most_similar(positive=["woman", "king"], negative=["man"], topn=1)
print("\nAnalogy result for 'king - man + woman':", analogy_result)

# 4. Check for a word in the vocabulary
word = "apple"
if word in glove_model:
    print(f"\n'{word}' is in the vocabulary.")
else:
    print(f"\n'{word}' is not in the vocabulary.")

# 5. Get word vector
vector = glove_model["apple"]
print("\nVector for 'apple':", vector)

Words similar to 'king':
queen: 0.6336
prince: 0.6197
monarch: 0.5900
kingdom: 0.5791
throne: 0.5606
ii: 0.5562
iii: 0.5503
crown: 0.5225
reign: 0.5217
kings: 0.5066

Similarity between 'king' and 'queen': 0.6336

Analogy result for 'king - man + woman': [('queen', 0.6713277101516724)]

'apple' is in the vocabulary.

Vector for 'apple': [-0.20842   -0.019668   0.063981  -0.71403   -0.21181   -0.59283
 -0.15316    0.044217   0.63289   -0.84821   -0.21129   -0.19763
  0.19029   -0.56226    0.27126    0.23782   -0.5189    -0.24518
  0.035243   0.096833   0.24898    0.71279    0.038279  -0.10514
 -0.4779    -0.39515   -0.27194   -0.44428    0.06113   -0.2318
 -0.35901   -0.18239    0.035507  -0.087719  -1.0816    -0.42521
  0.003224  -0.45991   -0.043462  -0.39031    0.519      0.21139
 -0.25527    1.1805    -0.19041   -0.12156    0.034186  -0.062316
  0.14421   -0.53366    0.47425   -0.4471     0.58047    0.43578
  0.1321    -0.095712  -0.37182   -0.013837   0.20601   -0.10099
  0.10685  

# Q1a) What is the size of the vocabulary formed from your training data?

In [140]:
import re
from collections import Counter

def tokenize(text):
    # Use simple whitespace tokenization and lowercasing
    return re.findall(r'\b\w+\b', text.lower())

# Count unique tokens
counter = Counter()
for example in train_dataset:
    tokens = tokenize(example["text"])
    counter.update(tokens)

# Size of the vocabulary
vocab_size = len(counter)
print("Vocabulary size:", vocab_size)

Vocabulary size: 16512


# Q1b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [142]:
counter = 0
for example in train_dataset:
    tokens = tokenize(example["text"])
    tokens = [token.lower() for token in tokens]
    for token in tokens:
        if token not in glove_model:
            counter += 1
            print(token)

print("Number of OOV words:", counter)


wisegirls
enrapturing
compleja
intelectualmente
retadora
orquídeas
originalidad
suspenser
obviation
gorefests
waydowntown
makmalbaf
exhilarate
nuttgens
petin
provocatuers
jirí
hubac
shapelessly
addessi
seldahl
wollter
mullinski
avventura
needn
narcotizing
precollegiate
sparklingly
superlarge
destinees
margolo
dominatrixes
scuzbag
idoosyncratic
flatula
denlopp
updatings
watstein
sappier
condensada
divertida
visualmente
entretenida
sorprenderá
exporing
capturou
sarcástica
divertida
demencial
predecesora
complejos
cadness
shagster
powaqqatsi
policiales
últimos
kaputschnik
kickass
travil
splittingly
aborbing
monkeyfun
bierbichler
crummles
bustingly
stultifyingly
deutchland
datedness
inhospitability
næs
hastier
estava
existência
papai
fato
inquestionável
talancón
drippiness
seldahl
oesn
montias
hotdogging
stumblings
birot
apesar
consegue
entreter
recoing
bizzarre
wollter
seldahl
alientation
sogginess
birot
mollà
involvingly
wifty
gerbosi
stuffiest
timewaster
strafings
debuter
histo
contando

# Q1c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

## Approach 1

In [145]:
# Load FastText model for OOV handling
fasttext_model = api.load("fasttext-wiki-news-subwords-300")
print("FastText model loaded successfully.")


FastText model loaded successfully.


In [153]:
import numpy as np

oov_words_vectors = {}

# Function to retrieve embedding, handling OOV words as well
def get_embedding(word, oov_vector_size=300):
    """
    Get the embedding of a word:
      - First try GloVe
      - Then FastText
      - Finally, generate a random vector if not found in either
    """
    
    if word in glove_model:
        return glove_model[word]

    elif word in fasttext_model:
        print(f"OOV word '{word}' found in FastText model.")
        return fasttext_model[word]

    else:
        print(f"Word '{word}' not found in either GloVe or FastText. Generating random embedding.")
        if word not in oov_words_vectors:
            oov_words_vectors[word] = np.random.rand(oov_vector_size)
        return oov_words_vectors[word]

# Example usage
words = ["computer", "datascientist", "qwertyuiop"]  
for word in words:
    embedding = get_embedding(word)
    if embedding is not None:
        print(f"Embedding for '{word}': {embedding[:5]}...")  # Displaying first 5 elements

Embedding for 'computer': [-0.27628   0.13999   0.098519 -0.64019   0.031988]...
Word 'datascientist' not found in either GloVe or FastText. Generating random embedding.
Embedding for 'datascientist': [0.3411798  0.15531629 0.5699988  0.69916882 0.27357539]...
Word 'qwertyuiop' not found in either GloVe or FastText. Generating random embedding.
Embedding for 'qwertyuiop': [0.40632766 0.21124225 0.97227036 0.67743276 0.98819265]...


## Approach 2